In [1]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('data/chat_history.db')
cursor = conn.cursor()

# Execute a query to retrieve all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the list of tables
print("Tables in the database:")
for table in tables:
    print(table[0])



Tables in the database:
conversation
conversation_record
<class 'AG.ConversationRecord'>
conversation_history_T
conversations_history_table_sqlalchemy


modal volume get db_data /chat_history.db . --force


In [2]:
import pandas as pd

# Read the 'conversation' table into a pandas DataFrame
df_conversation = pd.read_sql_query("SELECT * FROM conversations_history_table_sqlalchemy", conn)

# Display the first few rows of the DataFrame
#print(df_conversation.head())
df_conversation.head(10)


,message_id,session_id,role,content,created_at
0,c591e890-a4bd-4656-aabd-64aba1df8590,95bbb3dc-9356-4af8-b781-feda4afe65d5,user,hi sqlalchemy,2024-11-12 16:26:35.854878
1,319c5ed1-5995-4493-8d0c-ebf335e2cbb7,42e492e4-98f3-465f-8848-5afddd049d96,user,ok,2024-11-12 16:27:09.641403


In [7]:
df_conversation.to_csv('df_conversation.csv', index=False)


In [5]:
session_id = '65c01756-9252-4a9d-913c-d12b3dc5310d'

df_conversation_filt = pd.read_sql_query(
    f"SELECT * FROM conversation_history_T WHERE session_id = '{session_id}'", 
    conn
)
df_conversation_filt.head()

,id,session_id,role,content,created_at
0,aa701f1c-fa8c-4976-ad6d-b0f87744f572,65c01756-9252-4a9d-913c-d12b3dc5310d,user,hI,2024-11-10 13:45:06
1,afe3566a-e586-4fdb-96e4-cd94a6f2e01d,65c01756-9252-4a9d-913c-d12b3dc5310d,assistant,HHello! What’s been going on that brings you h...,2024-11-10 13:45:49
2,ae6294e3-b127-4dcc-a7e7-bea9dbf8fada,65c01756-9252-4a9d-913c-d12b3dc5310d,assistant,It seems like things have become overwhelming...,2024-11-11 12:00:01
3,3886dbea-d45f-47d5-a575-d97938182a31,65c01756-9252-4a9d-913c-d12b3dc5310d,assistant,That sounds quite frustrating.I'd love to exp...,2024-11-11 12:02:13


In [7]:
db_path = 'data/chat_history.db'
session_id = '65c01756-9252-4a9d-913c-d12b3dc5310d'

In [8]:
def fetch_conversation_history(session_id, db_path):
    conn = sqlite3.connect(db_path)
    query = """
    SELECT role, content FROM conversation_history_T
    WHERE session_id = ?
    ORDER BY created_at ASC
    """
    # Fetch all messages for the session
    df_conversation_filt = pd.read_sql_query(query, conn, params=(session_id,))
    conn.close()
    
    # Accumulate all messages into a single conversation history string
    conversation = ''
    for _, row in df_conversation_filt.iterrows():
        role, content = row['role'], row['content']
        message_text = f"{role.capitalize()}: {content}\n"
        conversation += message_text  # Accumulate in chronological order

    return conversation

In [12]:
fetch_conversation_history(session_id, db_path)

'User: hI\nAssistant: HHello! What’s been going on that brings you here today?\n'

In [10]:
fetch_conversation_history(session_id, db_path)

'User: hI\nAssistant: HHello! What’s been going on that brings you here today?\n'

In [5]:
# (Optional) Close the cursor if you're done with it
cursor.close()
